# Import

In [1]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import plotly.express as px
import json

## Import des données sur les modèles
Nous importons les modèles servant de base pour VApp. Par défaut, les modèles sont quantifiés en int4.

In [2]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


In [3]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

## Import data base de travail
Import de la base de données générée dans generation-score.ipynb.

In [4]:
data_project_score = pd.read_csv("hard-database/data_project_scoring.csv")
data_project_score['project_len_char'] = data_project_score['project_description'].apply(len)

In [5]:
data_project_score_select = data_project_score[data_project_score['project_score']>20]
data_project_score_select.head(3)

,Unnamed: 0,slug,url,name,name_initial,short_title,financers,financers_full,instructors,instructors_full,...,project_description,project_score,scoring_made,scoring_error,request_time_total,request_time_single,gpu,corrected_normalize_score,corrected_project_score,project_len_char
169,143389,eb7b-developper-un-projet-maas-formation-pour-...,/aides/eb7b-developper-un-projet-maas-formatio...,Développer un projet MaaS : pour un accès uniq...,NaN,NaN,['Cerema'],"[{'id': 58, 'name': 'Cerema', 'logo': 'https:/...",[],[],...,Revitalisation d'une zone humide,23,5,0,0.823581,0.164716,H100,0.96,4.6,32
178,144512,4b03-aide-pour-la-gestion-et-la-valorisation-d...,/aides/4b03-aide-pour-la-gestion-et-la-valoris...,Financer la gestion et la valorisation des cou...,NaN,NaN,['Conseil départemental des Landes'],"[{'id': 622, 'name': 'Conseil départemental de...",[],[],...,Revitalisation d'une zone humide,23,5,1,0.960288,0.160048,H100,0.96,4.6,32
584,73256,fdb3-faciliter-lemission-de-cautions-dans-le-c...,/aides/fdb3-faciliter-lemission-de-cautions-da...,Faciliter l'émission de cautions dans le cadre...,NaN,NaN,['Bpifrance'],"[{'id': 416, 'name': 'Bpifrance', 'logo': 'htt...",[],[],...,Revitalisation d'une zone humide,23,5,1,0.964971,0.160828,H100,0.96,4.6,32


## Usage de la fonction gen_prompt_question_mono_aide

In [6]:
# Selection d'un échantillion simple
row = data_project_score_select[['description_md','eligibility_md','project_description']].iloc[0]

aide_description,aide_eligibility,project_description = row['description_md'],row['eligibility_md'],row['project_description']

### Exemple d'usage
Pour générer des résumés plus pertinents, nous avons besoin d'un LLM plus "intelligent". Notre choix s'est orienté vers Qwen 2.5-14B, qui représente un bon compromis entre vitesse, performance et intelligence.

In [7]:
# Fonction disponible sous ./prompt_script/gen_prompt_question_mono_aide.py
from prompt_script.gen_prompt_question_mono_aide import gen_prompt_question_mono_aide

In [8]:
model = "qwen2.5:14b"
model_options = model_data[model]

request_options = {
    "num_ctx": 16384,
    "num_predict": 512
}

num_question = 3


prompt_system, prompt_user = gen_prompt_question_mono_aide(aide_description,aide_eligibility,project_description,num_question)

response = generate_ollama_request(
    prompt_system=prompt_system,
    response_format="json",
    prompt_user=prompt_user,
    ollama_api_url=ollama_api_url,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    model=model, 
    request_options= request_options,  # Default to None
    seed=0,
    )

### Convertion au format json
**Attention** : lors de l'usage du json il peut rester des reliquats de balise html, un post traitement peux aider

In [9]:
json_text = response['response']

# Remplacer les caractères HTML par une apostrophe
json_text = json_text.replace('&#39;', "'")

# Convertir la chaîne en dictionnaire Python
response_json = json.loads(json_text)

response_json

{'Q1': "Comment votre projet de revitalisation d'une zone humide est-il lié au concept Mobility-as-a-Service (MaaS) ou pourrait-il en tirer profit ?",
 'Q2': 'Quels objectifs spécifiques visitez-vous avec cette aide/subvention pour le développement du MaaS dans le cadre de la revitalisation des zones humides?',
 'Q3': "Avez-vous identifié les ressources documentaires essentielles ou certains projets emblématiques sur l'application du concept MaaS à un projet similaire, comme celui que vous prévoyez?"}

In [10]:
project_list = data_project_score_select.project_description.unique()

## Usage de la fonction gen_prompt_question_mono_aide

### Exemple d'usage
Nous avons remarqué que l'utilisation de plusieurs aides pour générer les questions était plus pertinente.
Nous avons donc implémenté une méthode capable d'intégrer trois questions basées sur un échantillon des trois meilleures aides trouvées.

Contrairement à la méthode "mono", nous n'intégrons pas l'éligibilité, ce qui permet d'avoir des questions plus générales et moins contraintes par les critères d'éligibilité. Cela aide l'utilisateur à mieux se concentrer sur son projet.

In [11]:
# Fonction disponible sous ./prompt_script/gen_prompt_question_multi_aide.py
from prompt_script.gen_prompt_question_multi_aide import gen_prompt_question_multi_aide

In [12]:
number_of_sub = 3

data_list_out = []

request_options = {
    "num_ctx": 16384,
    "num_predict": 512
}

for project_description in project_list:
    data_project_best = data_project_score_select[data_project_score_select['project_description']==project_description].sort_values("project_score")[-number_of_sub:]
    aide_description_list = data_project_best['description_md'].values
    sub1,sub2,sub3 = aide_description_list

    try:
        prompt_system, prompt_user = gen_prompt_question_multi_aide(aide_description_list,project_description,num_question)
        response = generate_ollama_request(
            prompt_system=prompt_system,
            response_format="json",
            prompt_user=prompt_user,
            ollama_api_url=ollama_api_url,
            bearer_token=ollama_bearer_token,
            model_options = model_options,  # Default to None
            model=model, 
            request_options= request_options,  # Default to None
            seed=0,
            )

        json_text = response['response']

        # Remplacer les caractères HTML par une apostrophe
        json_text = json_text.replace('&#39;', "'")

        # Convertir la chaîne en dictionnaire Python
        response_json = json.loads(json_text)

        q1 = response_json['Q1']
        q2 = response_json['Q2']
        q3 = response_json['Q3']

        data_list_out.append([project_description,sub1,sub2,sub3,q1,q2,q3])
        print('---------------')
        print(project_description)
    except Exception as error:
        print(error)

---------------
Revitalisation d'une zone humide
---------------
Entretient d'un vieux moulin
---------------
Réhabilitation d'une ancienne école en lieu dédié à la santé.
---------------
Voir fiche action PVD /ORT n°21 Après une première phase d’aménagement, la commune poursuit son action dans le cadre d’une 2ème phase d’aménagement. Cette prochaine étape concerne : - La mise en accessibilité du site aux personnes en situation de handicap et l’installation d’équipements en accès libre favorisant la pratique physique. - La protection de l’environnement par l’installation de sanitaires autonomes et la création d’un site O déchets - La création d'un parcours pédagogique
---------------
La commune dispose d'outils numériques qu'il est nécessaire d'optimiser et coordonner
---------------
Création d’un sentier thématique sur la forêt à Rieutord et sur le patrimoine à Usclades
---------------
Je souhaite refaire la voirie communal
---------------
Le projet consiste en l’aménagement d’un terr

In [13]:
data_question = pd.DataFrame(data_list_out,columns=['project_description','sub1','sub2','sub3','q1','q2','q3'])
data_question.to_csv('hard-database/data_question.csv')